# Alternating Direction Method of Multipliers (ADMM)

O objetivo deste tutorial é ilustrar a criação de estruturas para implementar um algoritmo de otimização, que usa modelos `JuMP` como parte de um algoritmo maior.
Nós vamos usar o [ADMM](https://en.wikipedia.org/wiki/Alternating_direction_method_of_multipliers) para resolver um problema de regressão linear com regularização $L^1$.

## Pacotes necessários

Este notebook requer os seguintes pacotes:

In [1]:
using JuMP
import Ipopt
import HiGHS

## Formulação

O caso clássico para ADMM é o de **otimização composta**, ou seja, problemas da forma
$$ \min_{x,z} f(x) + g(z) \quad \text{sujeito a} \quad z = Tx.$$

Estes problemas aparecem em diversas aplicações, e o interesse deles é duplo:
- A função objetivo pode ser separada em duas partes, uma que depende apenas de $x$ e outra que depende apenas de $z$;
- Cada uma das funções $f$ e $g$ pode ser otimizada de forma eficiente, mas, em geral, usando algoritmos diferentes.

Alguns exemplos de problemas compostos são:
- Regressão linear com regularização $L^1$ (Compressed Sensing, LASSO, Total Variation Denoising, etc.):
$$ \min_x \|Ax - b\|_2^2 + \lambda \|x\|_1$$
- Problemas de máxima veroossimilhança com uma priori: como $p(\theta|\text{data}) \propto \log p(\text{data}|\theta) + \log p(\theta)$, temos (ignorando uma constante)
$$ \max_\theta \log p(\text{data}|\theta) + \log p(\theta) $$
- Problemas de otimização estocástica em dois estágios:
$$ \min_x f_1(x) + \mathbb{E}[f_2(x,\xi)] $$
- Cálculo de interseções:
$$ x \in C_1 \quad \text{e} \quad x \in C_2 $$



O ADMM é um método iterativo para abordar problemas desta forma, e é baseado na seguinte ideia:
- O problema acima é equivalente a
$$ \min_{x,z} f(x) + g(z) + \frac{\rho}{2} \|z - Tx\|_2^2 \quad \text{sujeito a} \quad z = Tx,$$
onde $\rho > 0$ é um parâmetro.
- Dualizando a restrição com o multiplicador de Lagrange $\lambda = (\lambda_1, \ldots, \lambda_n)$ correto, obtemos
$$ \min_{x,z} f(x) + g(z) + \frac{\rho}{2} \|z - Tx\|_2^2 + \lambda^T(z - Tx).$$
Este problema é conhecido como **problema de Lagrangiano Aumentado**.

- A partir de uma condição inicial $(x_0, z_0, \lambda_0)$, o ADMM itera os seguintes passos:
$$ \begin{align*}
x^{k+1} &:= \arg\min_x f(x) + \frac{\rho}{2} \|z^k - Tx\|_2^2 + \lambda^T(z^k - Tx) \\
z^{k+1} &:= \arg\min_z g(z) + \frac{\rho}{2} \|z - Tx^{k+1}\|_2^2 + \lambda^T(z - Tx^{k+1}) \\
\lambda^{k+1} &:= \lambda^k + \rho (z^{k+1} - Tx^{k+1}).
\end{align*}$$

Vamos construir uma interface de forma que o usuário passe
- dois modelos JuMP, `f` e `g`, que representam as funções $f$ e $g$,
- dois vetores de variáveis, `x` e `z`, para representar as variáveis de decisão $x$ e $z$,
- a matriz `T`, e
- o parâmetro `ρ`.

In [2]:
function augmented_lagrangian!(f::JuMP.Model, x, T, ρ; first::Bool)
    m,n = size(T)

    # Modificando f:
    if first
        other = @variable(f, [1:m])
        error = @expression(f, other - T*x)
    else
        other = @variable(f, [1:n])
        error = @expression(f, x - T*other)
    end

    # Save information for ADMM
    f.ext[:orig_obj] = objective_function(f)
    f.ext[:ρ] = ρ
    f.ext[:var] = x
    f.ext[:other] = other
    f.ext[:error] = error

    return nothing
end

augmented_lagrangian! (generic function with 1 method)

In [3]:
function make_pair!(f, g, x, z, T, ρ)
    augmented_lagrangian!(f, x, T, ρ, first=true)
    augmented_lagrangian!(g, z, T, ρ, first=false)
    return nothing
end

make_pair! (generic function with 1 method)

In [4]:
function solve_proximal(m::JuMP.Model, λᵏ, otherᵏ)
    ρ = m.ext[:ρ]
    error = m.ext[:error]
    fix.(m.ext[:other], otherᵏ)
    @objective(m, Min, m.ext[:orig_obj] + 0.5ρ * sum(error.^2) + sum(λᵏ .* error))
    optimize!(m)
    return value.(m.ext[:var])
end

solve_proximal (generic function with 1 method)

In [5]:
using Random
Random.seed!(0)

A = randn(3, 10)
b = randn(3);

In [6]:
f = Model(Ipopt.Optimizer)
set_silent(f)
@variable(f, x[1:10])
@objective(f, Min, sum( (A*x .- b).^2 ) );

In [7]:
g = Model(HiGHS.Optimizer)
set_silent(g)
@variable(g, z[1:10])
@variable(g, absz[1:10])
@constraint(g, absz .>= z)
@constraint(g, absz .>= -z)
@objective(g, Min, sum( absz ) );

In [8]:
using LinearAlgebra
I10 = 1.0 * I(10)
ρ = 1.0
make_pair!(f, g, f[:x], g[:z], I10, ρ)

In [9]:
x_next = solve_proximal(f, zeros(10), zeros(10))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



10-element Vector{Float64}:
  0.1805764074366593
  0.35809697231813414
  0.12016691252601668
  0.15509628615117643
 -0.11240045494422057
 -0.30632299651558326
 -0.32079953593884103
 -0.08762970853491206
 -0.08139288517599796
 -0.2512313905384395

In [10]:
z_next = solve_proximal(g, zeros(10), x_next)

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
l_curr = zeros(10)
for i in 1:20
    curr_err = (I10*x_next - z_next)
    println("$i: $(norm(curr_err))")
    l_curr .-= ρ * curr_err
    x_next = solve_proximal(f, l_curr, z_next)
    z_next = solve_proximal(g, l_curr, x_next)
end

1: 0.6972353886899763
2: 0.6436953452629165
3: 0.5910537152121168
4: 0.3659829118449364
5: 0.26458376821572105
6: 0.24151275335881317
7: 0.15751403191216995
8: 0.04609751545708216
9: 0.07416003477687477
10: 0.08037853103147241
11: 0.07975683412398091
12: 0.07170738998717716
13: 0.0567043406472091
14: 0.03836400142643656
15: 0.020442907268347618
16: 0.012738675955534294
17: 0.014149740237798487
18: 0.021287569983069823
19: 0.03067791463466744
20: 0.024763969974625964


In [12]:
[x_next z_next l_curr]

10×3 Matrix{Float64}:
  0.0113319    0.0       -0.331667
  0.524761     0.524761  -1.0
  0.167878     0.167878  -1.0
  0.00603121   0.0       -0.594444
 -0.00694999   0.0        0.484254
 -0.621835    -0.621835   1.0
  0.00344862   0.0        0.965942
  0.0137036    0.0        0.43191
 -0.0021465    0.0        0.289382
 -9.7338e-5    0.0        0.978745

## Transformando em uma função

In [13]:
function admm(f, g, x, z, T, ρ; max_iter=100, tol=1e-6, verbose=0)
    make_pair!(f, g, x, z, T, ρ)
    λᵏ = zeros(length(z))
    zᵏ = zeros(length(z))
    for k in 1:max_iter
        xᵏ = solve_proximal(f, λᵏ, zᵏ)
        zᵏ = solve_proximal(g, λᵏ, xᵏ)
        errᵏ = T*xᵏ - zᵏ
        λᵏ .-= ρ * errᵏ
        if verbose > 0
            print("$k: $(norm(errᵏ))")
            if verbose > 1
                print("\n  x = $xᵏ\n  z = $zᵏ\n  λ = $λᵏ")
            end
            println()
        end
        if norm(errᵏ) < tol
            return (x=xᵏ, z=zᵏ, λ=λᵏ, k=k, err=norm(errᵏ))
        end
    end
end

admm (generic function with 1 method)

In [14]:
f = Model(Ipopt.Optimizer)
set_silent(f)
@variable(f, x[1:10])
@objective(f, Min, sum( (A*x .- b).^2 ) )

g = Model(HiGHS.Optimizer)
set_silent(g)
@variable(g, z[1:10])
@variable(g, absz[1:10])
@constraint(g, absz .>= z)
@constraint(g, absz .>= -z)
@objective(g, Min, sum( absz ) );

In [15]:
ret = admm(f, g, f[:x], g[:z], I10, ρ; max_iter=100, tol=1e-6, verbose=0)
[ret.x ret.z ret.λ]

10×3 Matrix{Float64}:
 -2.09582e-7   0.0       -0.343166
  0.496928     0.496928  -1.0
  0.155346     0.155346  -1.0
 -3.43461e-8   0.0       -0.601688
 -4.73088e-8   0.0        0.491978
 -0.663624    -0.663624   1.0
  4.7395e-7    0.0        0.965803
  2.07584e-7   0.0        0.420656
  4.74858e-8   0.0        0.292202
  1.86819e-7   0.0        0.981713

In [16]:
ret.k, ret.err

(71, 5.934794947901097e-7)

## Criando uma estrutura para armazenar o problema

In [17]:
struct ADMM
    f :: JuMP.Model
    g :: JuMP.Model
    x :: Vector{JuMP.VariableRef}
    z :: Vector{JuMP.VariableRef}
    T :: Matrix{Float64}
    ρ :: Float64
end

## Exercícios

- Em vez de modificar os modelos de $f$ e $g$, crie uma cópia usando `Base.copy`.
  Você também terá que usar `JuMP.set_optimizer` para dar um solver para cada cópia,
  pois `Base.copy` não copia o solver.
- Crie uma estrutura de dados `admm_data` para armazenar os dados do ADMM.
  Esta estrutura deve conter os seguintes campos:
  - `x`: o valor atual de $x$;
  - `z`: o valor atual de $z$;
  - `λ`: o valor atual de $\lambda$;
  - `ρ`: o valor atual de $\rho$;
  - `f`: o modelo de $f$;
  - `g`: o modelo de $g$;
  - `A`: a matriz $A$;
  - `solver_f`: o solver usado para resolver o problema de $f$;
  - `solver_g`: o solver usado para resolver o problema de $g$.